In [1]:
import hazelbean as hb
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib
import matplotlib.pyplot as plt

import os

In [4]:
ssp_list = ['ssp1','ssp2','ssp3','ssp4','ssp5']
ssps_list = ['ssp1','ssp2','ssp3','ssp4','ssp5']
rcp_list = ['26','45','60','60','85']
gcm_list = ['cc','gs','he','mr']

Changing = 'Climate+LU_Change'
modelnm = 'xgb3_Allinputs'
match_raster = '../Data/inputs/Base/country_ids.tif'

# ΔLU

Metric: change in total bio hectares cropland

In [8]:
lu = pd.read_csv('../Data/intermediate/LU_csvs/Cropmask_lulc_esa_2000.csv')
lu = lu.set_index('pixel_id')

hectares = pd.read_csv('../Data/intermediate/Crop_csvs/ha_per_pixel.csv')
hectares = hectares.set_index('pixel_id')

lu = lu.merge(hectares[['ha_per_pixel']], right_index=True,left_index=True,how='inner')
lu.head()

lu['ha_cropland'] = lu['%cropland'] * lu['ha_per_pixel']
LU_2000 = lu['ha_cropland'].sum()/1e9
LU_2000

2.5594917702605766

In [16]:
for rcp,ssp in zip(rcp_list,ssp_list):
    lu = pd.read_csv('../Data/intermediate/LU_csvs/Cropmask_lulc_RCP'+rcp+'_'+ssp+'.csv')
    lu = lu.set_index('pixel_id')

    hectares = pd.read_csv('../Data/intermediate/Crop_csvs/ha_per_pixel.csv')
    hectares = hectares.set_index('pixel_id')

    lu = lu.merge(hectares[['ha_per_pixel']], right_index=True,left_index=True,how='inner')
    lu.head()

    lu['ha_cropland'] = lu['%cropland'] * lu['ha_per_pixel']
    tot = lu['ha_cropland'].sum()/1e9
    dif = (tot - LU_2000 )*1e3
    print('{}: Total bio cropland: {:.2f}, ΔCropland(mio ha):{:.0f}'.format(ssp,tot,dif))

ssp1: Total bio cropland: 2.49, ΔCropland(mio ha):-68
ssp2: Total bio cropland: 2.70, ΔCropland(mio ha):142
ssp3: Total bio cropland: 2.76, ΔCropland(mio ha):196
ssp4: Total bio cropland: 2.73, ΔCropland(mio ha):166
ssp5: Total bio cropland: 2.79, ΔCropland(mio ha):226


# ΔClimate

In [32]:
climate_vars = ['Annual Mean Temperature', 'Mean Diurnal Range',
       'Isothermality', 'Temperature Seasonality',
       'Max Temperature of Warmest Month', 'Min Temperature of Coldest Month',
       'Temperature Annual Range', 'Annual Precipitation',
       'Precipitation of Wettest Month', 'Precipitation of Driest Month',
       'Precipitation Seasonality']

## Normalization

### Find extrema across all SSPs, GCMs and pixels

In [37]:
# Dictionaries with key = climate_vars , value = extrema
max_dict = dict.fromkeys(climate_vars,0)
min_dict = dict.fromkeys(climate_vars,0)

In [36]:
# Find global max for all climate vars
climate_csvs_dir = '../Data/intermediate/Climate_csvs'

for climate_var in climate_vars:
    global_max = 0
    for csv in os.listdir(climate_csvs_dir):
        if csv.endswith('.csv'):
            df = pd.read_csv(os.path.join(climate_csvs_dir,csv))
            df = df.set_index('pixel_id')
                             
            local_max = df[climate_var].max()
            global_max = max(global_max,local_max)
    max_dict[climate_var] = global_max
max_dict

{'Annual Mean Temperature': 350,
 'Mean Diurnal Range': 219,
 'Isothermality': 96.6964857388604,
 'Temperature Seasonality': 22656,
 'Max Temperature of Warmest Month': 533,
 'Min Temperature of Coldest Month': 280,
 'Temperature Annual Range': 735,
 'Annual Precipitation': 11339,
 'Precipitation of Wettest Month': 2760,
 'Precipitation of Driest Month': 738,
 'Precipitation Seasonality': 270}

In [ ]:
# Find global min for all climate vars

for climate_var in climate_vars:
    global_min = 0
    for csv in os.listdir(climate_csvs_dir):
        if csv.endswith('.csv'):
            df = pd.read_csv(os.path.join(climate_csvs_dir,csv))
            df = df.set_index('pixel_id')
                             
            local_min = df[climate_var].min()
            global_min = max(global_min,local_min)
    min_dict[climate_var] = global_min
min_dict

### Normalize

In [ ]:
TODO

## Make vectors of Δbioclim vars for each pixel (computationally crazy...)

# ΔMgt

# ΔDiet

# ΔPop

In [25]:
cur = pd.read_csv(os.path.join('../Data/intermediate/Population_csvs','2000.csv'))
cur = cur.set_index('pixel_id')
cur_pop = cur['gpw_population'].sum()/1e9
cur_pop

5.976296907318532

In [26]:
for csv in os.listdir('../Data/intermediate/Population_csvs'):
    if csv.startswith("adjusted"): 
        df = pd.read_csv(os.path.join('../Data/intermediate/Population_csvs',csv))
        df = df.set_index('pixel_id')
        tot = df['gpw_population'].sum()/1e9 - cur_pop
        print(csv[-8:-4]+' '+str(tot))

ssp5 2.271805529681468
ssp4 2.7466757166814686
ssp1 1.9736298066814681
ssp3 3.953020077681468
ssp2 2.2342945666814673


# ΔProd

In [17]:
def load_results(ssp,
                 Changing = 'All_Change',
                 modelnm = 'xgb3_Allinputs'):
    df = pd.read_csv('../Data/outputs/Model_results/'+Changing+'/'+modelnm+'/'+ssp+'_composite.csv')
    df = df.set_index('pixel_id')
    return df

In [22]:
col = 'Calories 2050'

cur_prod = load_results(ssp)['Calories 2000'].sum()/1e18
print('2000: Tot Prod: {:.1f}e18 cal'.format(cur_prod))

for ssp in ssp_list: 
    rez = load_results(ssp)
    tot_prod = rez['Calories 2050'].sum()/1e18 
    
    tot_Δprod = rez['ΔCalories'].sum()/1e18
    tot_Δprod_check = tot_prod - cur_prod
    
    print('{}: Tot Prod: {:.1f}e18 cal   ΔProd: {:.1f}/{:.1f}e18 cal'.format(ssp,tot_prod, tot_Δprod,tot_Δprod_check))

2000: Tot Prod: 16.8e18 cal
ssp1: Tot Prod: 15.4e18 cal   ΔProd: -1.4/-1.5e18 cal
ssp2: Tot Prod: 16.0e18 cal   ΔProd: -0.8/-0.8e18 cal
ssp3: Tot Prod: 15.5e18 cal   ΔProd: -1.4/-1.3e18 cal
ssp4: Tot Prod: 15.7e18 cal   ΔProd: -1.2/-1.2e18 cal
ssp5: Tot Prod: 17.0e18 cal   ΔProd: 0.2/0.2e18 cal
